<a href="https://colab.research.google.com/github/RiyaParikh0112/AQI-Prediction-Service/blob/main/0_Fetch_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
from datetime import datetime
from re import A
import requests
import os
import joblib
import pandas as pd
from dotenv import load_dotenv
load_dotenv()



def get_air_json(city_name):
    return requests.get(f'https://api.waqi.info/feed/{city_name}/?token=3f22aa832ccc530e82d0b9dd8eacd43c70016bee').json()['data']


def get_aqi_data(city_name):
    API_KEY = '3f22aa832ccc530e82d0b9dd8eacd43c70016bee'
    data = get_air_json(city_name)
    iaqi = data['iaqi']
    forecast = data['forecast']['daily']
    return [
        city_name,
        data['aqi'],                
        data['time']['s'][:10],     
        iaqi['h']['v'],
        iaqi['p']['v'],
        iaqi['pm10']['v'],
        iaqi['t']['v'],
        forecast['o3'][0]['avg'],
        forecast['o3'][0]['max'],
        forecast['o3'][0]['min'],
        forecast['pm10'][0]['avg'],
        forecast['pm10'][0]['max'],
        forecast['pm10'][0]['min'],
        forecast['pm25'][0]['avg'],
        forecast['pm25'][0]['max'],
        forecast['pm25'][0]['min']
    ]

def get_aqi_df(data):
    col_names = [
        'city',
        'aqi',
        'date',
        'iaqi_h',
        'iaqi_p',
        'iaqi_pm10',
        'iaqi_t',
        'o3_avg',
        'o3_max',
        'o3_min',
        'pm10_avg',
        'pm10_max',
        'pm10_min',
        'pm25_avg',
        'pm25_max',
        'pm25_min'
    ]

    new_data = pd.DataFrame(
        data,
        columns=col_names
    )
    new_data.date = new_data.date.apply(timestamp)

    return new_data


def timestamp(x):
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)





In [41]:
cities = ['Beijing','Kyiv','Tokyo','Kyoto','Kobe','Osaka','Malmo','Yokohama','Hiroshima','Sapporo','Nagasaki']

data_aqi = [get_aqi_data(city) for city in cities]


In [43]:
df_aqi = get_aqi_df(data_aqi)

df_aqi

,city,aqi,date,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,o3_avg,o3_max,o3_min,pm10_avg,pm10_max,pm10_min,pm25_avg,pm25_max,pm25_min
0,Beijing,35,1670976000000,18.00,1033.0,35,-6.0,3,17,1,29,46,7,93,138,30
1,Kyiv,5,1666137600000,88.23,1017.6,2,14.9,21,29,12,16,19,10,54,64,34
2,Tokyo,69,1670976000000,99.30,1004.6,69,7.2,6,14,1,16,23,9,54,67,30
3,Kyoto,61,1670976000000,63.70,1013.5,17,10.0,8,18,1,11,17,1,28,45,13
4,Kobe,80,1670976000000,54.00,1015.0,51,13.0,9,18,1,13,19,1,40,56,14
5,Osaka,80,1670976000000,50.00,1014.0,43,11.0,7,17,1,15,19,1,46,65,13
6,Malmo,18,1670889600000,69.50,1005.3,12,-2.0,21,27,15,10,21,4,30,66,8
7,Yokohama,38,1670976000000,89.00,1006.0,11,9.0,6,13,1,14,19,10,45,61,30
8,Hiroshima,76,1670976000000,39.00,1010.8,34,10.7,10,18,4,15,19,7,19,21,13
9,Sapporo,38,1670976000000,81.00,998.2,14,1.4,16,17,15,1,1,1,4,13,1


In [39]:
df_aqi.to_csv('aqi_city.csv')